# **Cortex Predicitve Analytics Fundraising Scenario**

**ROUND 1: Predict donation amount**

Daniel Salvador Cázares García A01197517

# **Initial set up**

## **Connect to SASPy**

**0. Connect to Google Drive**

In [ ]:
my_folder = "/content/drive/MyDrive"

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir(my_folder)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Concentración IA & DS/Reto SAS


**1. Python & Java version**

In [ ]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [ ]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [ ]:
!pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 7.0 MB/s 
  Created wheel for saspy: filename=saspy-4.4.1-py3-none-any.whl size=9937304 sha256=83664c32534fa517f81c9e500bed496ce4f836817ed21d64d2e228a337c4b352
  Stored in directory: /root/.cache/pip/wheels/0a/f6/ab/f7173c8a824856c66f6fd50194bb8fd3ef93fb4f1824be8d0c
Successfully built saspy


**3. Configuration file**

In [ ]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 2
'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4. Create .authinfo file**

In [ ]:
%%writefile .authinfo
oda user your-user@mail.com password your-password

Overwriting .authinfo


In [ ]:
!cp .authinfo ~/.authinfo

**5. Establish Connection (Need to do this step each time you use SASPy)**

In [ ]:
import saspy
sas_session = saspy.SASsession(cfgfile="/content/drive/MyDrive/sascfg_personal.py")
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 355



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/Concentración IA & DS/Reto SAS/sascfg_personal.py
WORK Path             = /saswork/SAS_work86EA0000A64F_odaws01-usw2-2.oda.sas.com/SAS_workA21D0000A64F_odaws01-usw2-2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 42575


## Connect to Cortex Data Sets

In [ ]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


5                                                          The SAS System                      Friday, December  2, 2022 02:23:00 AM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         
27             libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
28         
29         
30         
31         ods html5 (id=saspy_internal) close;ods listing;
32         

6                                                          The SAS System                      Friday, December  2, 2022 02:23:00 AM

33         


## Transform SAS dataset to dataframe

In [ ]:
import pandas as pd

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd1',
libref='cortex'
)

In [ ]:
merge_data = pd.merge(data1, data2, on=["ID"],how="right")
#merge_data['ID'] = merge_data['ID'].astype('Int64')
merge_data.head()

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,GaveThisYear,AmtThisYear
0,2000001.0,ROMMES,RODNEY,0.0,25.0,107200.0,University / College,City,2.0,0.0,...,1.0,2.0,2.0,1010.0,10.0,1000.0,0.0,0.0,0.0,0.0
1,2000002.0,RAMIREZ,SHARON,1.0,38.0,15800.0,High School,Rural,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,2000003.0,TSOSIE,KAREN,1.0,37.0,57400.0,University / College,Rural,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,2000004.0,LEE,MARY,1.0,78.0,23700.0,High School,Rural,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,2000005.0,HUMPHRES,ANGIE,1.0,34.0,71900.0,University / College,Rural,8.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


***
# **Data**

In [ ]:
target_data = merge_data

In [ ]:
target_data.describe()

,ID,Woman,Age,Salary,SeniorList,NbActivities,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,GaveThisYear,AmtThisYear
count,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,336334.000000,336334.000000,336334.000000,336334.000000,336334.000000,336334.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,2.500000e+06,0.516936,46.366170,65531.654000,4.582967,0.561413,0.560556,3.039636,1.666882,4.474148,104.454055,43.130251,85.897932,0.122101,7.673015,0.122232,7.741080
std,2.886753e+05,0.499713,18.945324,61051.122343,3.325844,0.996803,1.054027,2.120722,1.052090,2.588119,303.760477,150.249476,283.581066,0.327403,82.224854,0.327554,83.110552
min,2.000001e+06,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,10.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000
25%,2.250001e+06,0.000000,30.000000,20700.000000,2.000000,0.000000,0.000000,1.000000,1.000000,2.000000,20.000000,15.000000,20.000000,0.000000,0.000000,0.000000,0.000000
50%,2.500000e+06,1.000000,46.000000,44000.000000,4.000000,0.000000,0.000000,2.000000,1.000000,4.000000,40.000000,20.000000,30.000000,0.000000,0.000000,0.000000,0.000000
75%,2.750000e+06,1.000000,61.000000,91200.000000,7.000000,1.000000,1.000000,4.000000,2.000000,6.000000,90.000000,30.000000,75.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000e+06,1.000000,90.000000,250000.000000,10.000000,10.000000,16.000000,10.000000,10.000000,10.000000,15150.000000,10000.000000,10000.000000,1.000000,10000.000000,1.000000,10000.000000


### Variables

In [ ]:
pd.crosstab(target_data['Woman'],  'count')

col_0,count
Woman,
0.0,483064
1.0,516936


In [ ]:
pd.crosstab(target_data['Education'],  'count')

col_0,count
Education,
Elementary,29580
High School,322516
University / College,647904


In [ ]:
pd.crosstab(target_data['City'],  'count')

col_0,count
City,
City,332657
Downtown,156694
Rural,197978
Suburban,312671


### Missing Values

In [ ]:
target_data.isna().sum()

ID                   0
LastName            23
FirstName            4
Woman                0
Age                  0
Salary               0
Education            0
City                 0
SeniorList           0
NbActivities         0
Referrals            0
Recency         663666
Frequency       663666
Seniority       663666
TotalGift       663666
MinGift         663666
MaxGift         663666
GaveLastYear         0
AmtLastYear          0
GaveThisYear         0
AmtThisYear          0
dtype: int64

***
# **Model**

## Data Preparation

Deal with missing values

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
ohe.fit(target_data[['Education', 'City']])
ohe_df = pd.DataFrame(data=ohe.transform(target_data[['Education', 'City']]), columns=ohe.get_feature_names_out())

target_data = target_data.join(ohe_df)

In [ ]:
target_data['Recency'] = target_data['Recency'].fillna((target_data['GaveLastYear'] == 1.0).map({True: 1.0}))
target_data['Recency'] = target_data['Recency'].fillna(value=0)
target_data['Frequency'] = target_data['Frequency'].fillna((target_data['GaveLastYear'] == 1.0).map({True: 1.0}))
target_data['Frequency'] = target_data['Frequency'].fillna(value=0)

In [ ]:
target_data['MinGift'] = target_data['MinGift'].fillna(target_data['AmtLastYear'])  
target_data['MaxGift'] = target_data['MaxGift'].fillna(target_data['AmtLastYear'])
target_data['TotalGift'] = target_data['TotalGift'].fillna(target_data['AmtLastYear'])

In [ ]:
target_data.isna().sum()

ID                                     0
LastName                              23
FirstName                              4
Woman                                  0
Age                                    0
Salary                                 0
Education                              0
City                                   0
SeniorList                             0
NbActivities                           0
Referrals                              0
Recency                                0
Frequency                              0
Seniority                         663666
TotalGift                              0
MinGift                                0
MaxGift                                0
GaveLastYear                           0
AmtLastYear                            0
GaveThisYear                           0
AmtThisYear                            0
Education_Elementary                   0
Education_High School                  0
Education_University / College         0
City_City       

## Data Partition

In [ ]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(target_data, train_size=0.8, random_state=42) 

train.sample(5)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,AmtLastYear,GaveThisYear,AmtThisYear,Education_Elementary,Education_High School,Education_University / College,City_City,City_Downtown,City_Rural,City_Suburban
141528,2141529.0,ROSBOROUGH,WILLIAM,0.0,56.0,147500.0,University / College,Rural,1.0,0.0,...,0.0,1.0,10.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
347896,2347897.0,REDWINE,DUANE,0.0,35.0,189100.0,University / College,Suburban,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
35000,2035001.0,SANFORD,TERESA,1.0,81.0,59000.0,High School,City,2.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
449454,2449455.0,FREELON,MICHAEL,0.0,66.0,11300.0,High School,Rural,1.0,0.0,...,20.0,1.0,25.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
82979,2082980.0,BODICK,PEDRO,0.0,62.0,30400.0,University / College,Downtown,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [ ]:
target_data.columns

Index(['ID', 'LastName', 'FirstName', 'Woman', 'Age', 'Salary', 'Education',
       'City', 'SeniorList', 'NbActivities', 'Referrals', 'Recency',
       'Frequency', 'Seniority', 'TotalGift', 'MinGift', 'MaxGift',
       'GaveLastYear', 'AmtLastYear', 'GaveThisYear', 'AmtThisYear',
       'Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban'],
      dtype='object')

In [ ]:
selected_cols = ['Woman', 'Age', 'Salary',
       'NbActivities', 'Referrals', 'Recency',
       'Frequency', 'TotalGift', 'MinGift', 'MaxGift',
       'GaveLastYear', 'AmtLastYear',
       'Education_Elementary', 'Education_High School',
       'Education_University / College', 'City_City', 'City_Downtown',
       'City_Rural', 'City_Suburban']

In [ ]:
X_train = train[selected_cols] 
Y_train = train['AmtThisYear']

X_valid = validation[selected_cols] 
Y_valid = validation['AmtThisYear']

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train = StandardScaler().fit_transform(X_train)
X_valid = StandardScaler().fit_transform(X_valid)

## Training

In [ ]:
import numpy as np
from sklearn import metrics

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=50, random_state=42, criterion="squared_error")
rfr.fit(X_train, Y_train)
rfr_pred = rfr.predict(X_valid)

In [ ]:
#MAE
print(metrics.mean_absolute_error(Y_valid, rfr_pred))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid, rfr_pred)))

15.045402981320153
95.54810437896333


### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DT_model = DecisionTreeRegressor(max_depth = 5).fit(X_train, Y_train)
DT_predict = DT_model.predict(X_valid)

In [ ]:
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

13.213324118264444
88.93845659467834


### XGBoost

In [ ]:
import xgboost as xg

xgb_r = xg.XGBRegressor(objective ='reg:linear')
xgb_r.fit(X_train, Y_train)
xgb_pred = xgb_r.predict(X_valid)

[01:14:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
#MAE
print(metrics.mean_absolute_error(Y_valid, xgb_pred))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid, xgb_pred)))

13.11400527150944
88.96656466326226


# **Results**

## Scoring Data

In [ ]:
data3 = sas_session.sasdata2dataframe(
table='score_rd1',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)

In [ ]:
scoring_data = pd.merge(data3, data4, on=["ID"],how="right")
#scoring_data.head()

## Data Preparation

In [ ]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(scoring_data[['Education', 'City']])
ohe_df = pd.DataFrame(data=ohe.transform(scoring_data[['Education', 'City']]), columns=ohe.get_feature_names_out())

scoring_data = scoring_data.join(ohe_df)

In [ ]:
scoring_data['Recency'] = scoring_data['Recency'].fillna((scoring_data['GaveLastYear'] == 1.0).map({True: 1.0}))
scoring_data['Recency'] = scoring_data['Recency'].fillna(value=0)
scoring_data['Frequency'] = scoring_data['Frequency'].fillna((scoring_data['GaveLastYear'] == 1.0).map({True: 1.0}))
scoring_data['Frequency'] = scoring_data['Frequency'].fillna(value=0)

In [ ]:
scoring_data['MinGift'] = scoring_data['MinGift'].fillna(scoring_data['AmtLastYear'])  
scoring_data['MaxGift'] = scoring_data['MaxGift'].fillna(scoring_data['AmtLastYear'])
scoring_data['TotalGift'] = scoring_data['TotalGift'].fillna(scoring_data['AmtLastYear'])

In [ ]:
X_end = scoring_data[selected_cols]
X_end = StandardScaler().fit_transform(X_end)

## Training

In [ ]:
#final_pred = DT_model.predict(X_end)
final_pred = xgb_r.predict(X_end)

## Export Results to CSV

In [ ]:
scoring_data['Prediction'] = final_pred
scoring_data.sort_values(by=['Prediction'], inplace=True, ascending=False)
scoring_data['ID'] = scoring_data['ID'].astype('Int64')
scoring_data.head()

,ID,GaveLastYear,AmtLastYear,LastName,FirstName,Woman,Age,Salary,Education,City,...,MinGift,MaxGift,Education_Elementary,Education_High School,Education_University / College,City_City,City_Downtown,City_Rural,City_Suburban,Prediction
723373,2723374,0.0,0.0,SAIZ,EILEEN,1.0,61.0,14200.0,High School,Suburban,...,10.0,10.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6724.588379
143762,2143763,1.0,50.0,HINKLEMAN,NATASHA,1.0,28.0,123500.0,University / College,City,...,50.0,50.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1863.729004
146477,2146478,0.0,0.0,LI,DANIELLE,1.0,36.0,86200.0,High School,Downtown,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1624.048950
369019,2369020,0.0,0.0,RAYBOULD,KAY,1.0,46.0,235400.0,High School,Rural,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1602.081055
165587,2165588,0.0,0.0,BOULET,JESSICA,1.0,77.0,39400.0,High School,Suburban,...,20.0,40.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1570.738159


In [ ]:
# Define your cutoff and choose a number of rows to submit to the leaderboard

file_name = 'R1 XGB.csv'
NB = 60000
submission = scoring_data[['ID']].head(NB)
submission.to_csv(file_name, index=False, header=False)